In [102]:
#importing all the necessary libraries
import numpy as np
import os

from sklearn.model_selection import train_test_split
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import AllPairs, OneAgainstRest
from qiskit.aqua.utils.dataset_helper import get_feature_dimension
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
SEED = 1430


In [103]:
# Read text File
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        print(f.read())
        

path = r"C:\Users\thomas.agnew\Desktop\QM workshop\poster\pneumonia_dataset\train_datasets" # Folder Path
os.chdir(path) # Change the directory
raw_data_train = []
for file in os.listdir(): # iterate through all file
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}\{file}"
  
        # call read text file function
        raw_data_train.append(np.genfromtxt(fname=file_path))
raw_data_train = np.concatenate((raw_data_train))
# raw_data_train = np.concatenate((raw_data_train[0:10],raw_data_train[50:60],raw_data_train[100:110]))
    # use the above line if you want to take only slices of the total data set
print(len(raw_data_train))
        
path = r"C:\Users\thomas.agnew\Desktop\QM workshop\poster\pneumonia_dataset\test_datasets" # Folder Path
os.chdir(path) # Change the directory
raw_data_test = []
for file in os.listdir(): # iterate through all file
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}\{file}"
  
        # call read text file function
    raw_data_test.append(np.genfromtxt(fname=file_path))
raw_data_test = np.concatenate((raw_data_test))
# raw_data_test = np.concatenate((raw_data_test[0:10],raw_data_test[50:60],raw_data_test[100:110]))
# use the above line if you want to take only slices of the total data set
print(len(raw_data_test))

raw_data_total = np.concatenate((raw_data_train, raw_data_test))
print(len(raw_data_total))

30
30
60


In [104]:
#processing the data: setting class names to A, B, and C, and splitting into a list of data and a list of classes. 

data_total=[]
class_total=[]
for i in range(len(raw_data_total)):
    if int(raw_data_total[i][-1])==0:
        class_total.append('A')
    elif int(raw_data_total[i][-1])==1:
        class_total.append('B')
    elif int(raw_data_total[i][-1])==2:
        class_total.append('C')
    data_total.append(raw_data_total[i][0:7])
    
# split our data into train and test sets using an 80-20 split 
x_train, x_test, y_train, y_test= train_test_split(
    data_total, class_total, test_size=0.2, random_state=SEED, stratify=class_total)

print(f'Size of the train set: {len(x_train)}\nSize of the test set: {len(x_test)}')
print((x_train[0]))
print(y_train)

Size of the train set: 48
Size of the test set: 12
[ 26728.28174304   4284.32453881 -12128.52916276  -2382.48791138
   2817.90216545  -3610.20727148  -5799.5155579 ]
['A', 'A', 'B', 'C', 'A', 'C', 'A', 'A', 'A', 'A', 'C', 'B', 'B', 'C', 'B', 'C', 'B', 'A', 'C', 'C', 'B', 'C', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'C', 'A', 'B', 'A', 'B', 'C', 'A', 'A', 'A', 'C', 'B', 'C', 'A', 'C', 'C', 'C', 'B']


In [105]:
# QSVM simulations using more than 2 classes require the data to be input as a dictionary. 
# the code below is for constructing the dictionary. 

Dict_train = {}
Dict_train.setdefault("A",[]) #initiallized a dictionary key 'A' corresponding to an empty list
Dict_train.setdefault("B",[])
Dict_train.setdefault("C",[])

Dict_test = {}
Dict_test.setdefault("A",[])
Dict_test.setdefault("B",[])
Dict_test.setdefault("C",[])

for i in range(len(y_train)):
    Dict_train[y_train[i]].append(x_train[i]) #appending to existing key
for i in range(len(y_test)):
    Dict_test[y_test[i]].append(x_test[i])

temp = [Dict_test[k] for k in Dict_test]
total_array = np.concatenate(temp)   

In [ ]:
aqua_globals.random_seed = 10598

for i in range(1,11,1): #iterates over number of quantum circuit repetitions.
    print("Number of repetitions=",i)
    backend = BasicAer.get_backend('qasm_simulator') #we use the qasm simulator
    feature_map = ZZFeatureMap(feature_dimension=get_feature_dimension(Dict_train),
                               reps=i, entanglement='linear') # creating the feature map.
    qc = feature_map.assign_parameters(Dict_train["A"][0])

    svm = QSVM(feature_map, Dict_train, Dict_test, total_array,
               multiclass_extension=OneAgainstRest()) #could also try OneVersusOne()

    quantum_instance = QuantumInstance(backend, shots=1024,
                                       seed_simulator=aqua_globals.random_seed,
                                       seed_transpiler=aqua_globals.random_seed)

    result = svm.run(quantum_instance) #running the simulation
    for k,v in result.items():
        print(f'{k} : {v}')


Number of repetitions= 1
testing_accuracy : 0.08333333333333337
test_success_ratio : 0.08333333333333337
predicted_labels : [2 1 0 2 0 0 0 0 1 0 1 1]
predicted_classes : ['C', 'B', 'A', 'C', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'B']
Number of repetitions= 2
testing_accuracy : 0.5
test_success_ratio : 0.5
predicted_labels : [0 0 1 2 2 2 2 1 2 2 1 2]
predicted_classes : ['A', 'A', 'B', 'C', 'C', 'C', 'C', 'B', 'C', 'C', 'B', 'C']
Number of repetitions= 3
testing_accuracy : 0.33333333333333337
test_success_ratio : 0.33333333333333337
predicted_labels : [1 0 0 1 1 0 0 1 0 1 0 0]
predicted_classes : ['B', 'A', 'A', 'B', 'B', 'A', 'A', 'B', 'A', 'B', 'A', 'A']
Number of repetitions= 4
testing_accuracy : 0.41666666666666663
test_success_ratio : 0.41666666666666663
predicted_labels : [2 1 2 0 1 1 0 2 1 1 2 2]
predicted_classes : ['C', 'B', 'C', 'A', 'B', 'B', 'A', 'C', 'B', 'B', 'C', 'C']
Number of repetitions= 5
